<a href="https://colab.research.google.com/github/seanrubin/Predicting-Popularity-of-Spotify/blob/main/SpotifySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark py4j
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=b9a524b111184cfec9dc4a0022fbb631eb34aa4f8b3265ae39184649f3afc538
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [861 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubunt

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
#Spotify_df = spark.read.format("csv").option("header", "true").load("sample_data/spotify_data_trimmed2.csv")
#Spotify_df.show(5)
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Verify the file path
file_path = "spotify_data_trimmed2.csv"  # Adjust if necessary
if not os.path.exists(file_path):
    print(f"Error: File not found at {file_path}")
else:
    Spotify_df = spark.read.format("csv").option("header", "true").load(file_path)
    Spotify_df.show(5)

+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|3nqQXoyQOWXiESFLl...|Sam Smith;Kim Petras|Unholy (feat. Kim...|Unholy (feat. Kim...|       100|     156943|   False|       0.714| 0.472|  2|  -7.375|   1|     0.0864|       0.013|        4.51e-06|   0.266|  0.238|131.121|             4|      dance|


In [ ]:
Spotify_df.createOrReplaceTempView('Spotify')
Spotify_df.show()

+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|3nqQXoyQOWXiESFLl...|Sam Smith;Kim Petras|Unholy (feat. Kim...|Unholy (feat. Kim...|       100|     156943|   False|       0.714| 0.472|  2|  -7.375|   1|     0.0864|       0.013|        4.51e-06|   0.266|  0.238|131.121|             4|      dance|


In [ ]:
a = """
SELECT
  track_name AS Track_Name,
  CAST(popularity as INT) AS Popularity
FROM Spotify
WHERE Popularity >=71
ORDER BY Popularity DESC
"""
spark.sql(a).show()

+--------------------+----------+
|          Track_Name|Popularity|
+--------------------+----------+
|Unholy (feat. Kim...|       100|
|Quevedo: Bzrp Mus...|        99|
|     I'm Good (Blue)|        98|
|          La Bachata|        98|
|     Me Porto Bonito|        97|
|    Tití Me Preguntó|        97|
|              Efecto|        96|
| Under The Influence|        96|
|     I Ain't Worried|        96|
|           As It Was|        95|
|       Ojitos Lindos|        95|
|         Moscow Mule|        94|
|       Glimpse of Us|        94|
|             CUFF IT|        93|
|     Sweater Weather|        93|
|        Another Love|        93|
|            Neverita|        93|
|            PROVENZA|        93|
|           As It Was|        92|
|Left and Right (F...|        92|
+--------------------+----------+
only showing top 20 rows



In [ ]:
b = """
SELECT
  track_genre As Genre,
  count(track_genre) AS Genre_Count
FROM Spotify
WHERE CAST(popularity as INT)>=71
GROUP BY track_genre
ORDER BY Genre_Count DESC
"""
spark.sql(b).show()

+-----------------+-----------+
|            Genre|Genre_Count|
+-----------------+-----------+
|            dance|        239|
|            k-pop|        185|
|         alt-rock|        147|
|           latino|        131|
|          electro|        119|
|        indie-pop|        112|
|          hip-hop|        109|
|              edm|        103|
|              pop|         93|
|        hard-rock|         92|
|          country|         68|
|             rock|         68|
|              emo|         62|
|             folk|         57|
|            blues|         56|
|          british|         55|
|            disco|         51|
|      alternative|         50|
|singer-songwriter|         48|
|            latin|         45|
+-----------------+-----------+
only showing top 20 rows



In [ ]:
d = """
SELECT
  AVG(popularity) AS Avg_Popularity,
  track_genre,
  artists,
  album_name,
  duration_ms,
  explicit,
  danceability,
  energy,
  key,
  loudness,
  mode,
  speechiness,
  acousticness,
  instrumentalness,
  liveness,
  valence,
  tempo,
  time_signature,
  track_genre
FROM Spotify
WHERE popularity>=71
GROUP BY track_genre, artists, album_name, duration_ms, explicit, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature
LIMIT 1
"""
spark.sql(d).show()

+--------------+-----------+------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+------+--------------+-----------+
|Avg_Popularity|track_genre|     artists|album_name|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence| tempo|time_signature|track_genre|
+--------------+-----------+------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+------+--------------+-----------+
|          86.0|        pop|Taylor Swift|  folklore|     261922|   False|       0.532| 0.623|  5|  -9.208|   1|     0.0331|       0.538|        7.28e-05|  0.0925|  0.403|89.937|             4|        pop|
+--------------+-----------+------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+------+---